# Data Visualization (2017/18)

## Solutions for Assignment 3 - Visualizing multivariate data 

Presented by Group 25: 
- Ganesh
- Chaitra

Date: 06.12.2018

## Setup

In [1]:
import pandas as pd
import numpy as np

# import bokeh 
from bokeh.plotting import figure, show, Figure
from bokeh.models import ColumnDataSource, Label, HoverTool
from bokeh.models.glyphs import Text
from bokeh.palettes import Spectral3
from bokeh.layouts import row, column, gridplot

# tell bokeh to show the figures in the notebook
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

Load data stored in bokeh:

In [2]:
from bokeh.sampledata.autompg import autompg
from bokeh.sampledata.iris import flowers
#flowers.head(100)
#autompg.head(392)
#autompg.groupby(['name']).count()

## Helpful functions

Group a dataframe according to a variable (species) and compute some statistics for a second variable (petal_width).

In [3]:
flowers.groupby(['species']).petal_width.describe()

,count,mean,std,min,25%,50%,75%,max
species,,,,,,,,
setosa,50.0,0.246,0.105386,0.1,0.2,0.2,0.3,0.6
versicolor,50.0,1.326,0.197753,1.0,1.2,1.3,1.5,1.8
virginica,50.0,2.026,0.274650,1.4,1.8,2.0,2.3,2.5


Find unique values and count them in categorical variable.

In [4]:
flowers.species.unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [5]:
flowers.species.value_counts()

versicolor    50
virginica     50
setosa        50
Name: species, dtype: int64

Use numpy to compute a histogram for quantitative data. See the docu for further information and how to work with the output.

In [6]:
np.histogram(flowers.petal_width)

(array([41,  8,  1,  7,  8, 33,  6, 23,  9, 14]),
 array([0.1 , 0.34, 0.58, 0.82, 1.06, 1.3 , 1.54, 1.78, 2.02, 2.26, 2.5 ]))

## Exercise 1 a): Customize a scatterplot chart

The following code skeleton renders a scatterplot. Customize the chart to your liking. Think for example of many data points. 

This is meant to be a very quick exercise to demonstrate the concept for the following two charts.

Requirements:
- **Parameters**: The function accepts (at least) the following parameters:
    - **source**: a pandas DataFrameObject or bokeh ColumnDataSource that holds the data
    - **x**: variable (name as string) to be represented on the x-axis
    - **y**: variable (name as string) to be represented on the y-axis
- **Calling the scatterplot**: The function is a class method of Figure and can be called as follows
```python
p = figure()
p.scatter( data, x, y )
```
This is already setup in the code skeleton below.

**<font color="deeppink">Update code</font>**

In [7]:
def scatter( self, source, x, y,**kwargs ):
    # access the figure using the self variable
    self.circle( source=source, x=x, y=y, **kwargs)
    source = ColumnDataSource(data=dict(weight=autompg['weight'].tolist(), hp=autompg['hp'].tolist(), name=autompg['name'].tolist()))
    
    #label = Label( x=50, y=50, x_units='screen', y_units='screen', render_mode='css' )
    #self.add_layout(label)

# add the function as class method to Figure    
Figure.scatter = scatter

**<font color="deeppink">Check</font>** that your code is working:

In [8]:
hover = HoverTool(tooltips=[
    ('name', '@name'),
    ("(weight,hp)", "(@weight, @hp)"),
])
p = figure( plot_width=300, plot_height=300, tools=[hover] )
p.scatter( source=autompg, x='weight', y='hp', legend="HP vs Weight")
p.xaxis.axis_label="Weight"
p.yaxis.axis_label="HP"
show(p)


**<font color="deeppink">Test cases</font>**: Give three scenarios that need testing (bullet points, no implementation required). Think of scenarios where your code may fail.
- Test case 1 - Null values in the respective weight or hp columns will yield to wrong scatter plots.
- Test case 2 - Wrong scatter plots when name column does not have unique values.
- Test case 3 - Code will fail when there is no check on the datatypes for columns. Example: if HP has a char value, scatter plot fails.

## Exercise 1 b): Implement a boxplot chart

Requirements:
- **Parameters**: The function accepts (at least) the following parameters:
    - **source**: a pandas DataFrameObject that holds the data
    - **x**: variable (name as string) to be represented on the x-axis
    - **y**: variable (name as string) to be represented on the y-axis
- **Orientation**: Provide boxplots with horizontal and vertical orientation (call them hboxplot and vboxplot).
- **Calling the boxplot**: The function is a class method of Figure and can be called as follows
```python
p = figure()
p.vboxplot( data, x, y )
```
This is already setup in the code skeleton below.

Hints:
- A Bokeh sample implementation can be found here: [Boxplot](https://bokeh.pydata.org/en/latest/docs/gallery/boxplot.html)
- Adapt this implementation to work on the target variable only. See code below to get started.

**<font color="deeppink">Implement</font>**

In [9]:
def vboxplot( self, source, x, y, **kwargs ):
    if not isinstance(source, pd.DataFrame ):
        raise TypeError("source has to be a pandas DataFrame.")
        
    flowerList = list(source[x].unique())
    df = pd.DataFrame(source)

    groups = df.groupby(x)
    q1 = groups.quantile(q=0.25)
    q2 = groups.quantile(q=0.5)
    q3 = groups.quantile(q=0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr
    
    def outliers(group):
        flower = group.name
        return group[(group[y] > upper.loc[flower][y]) | (group[y] < lower.loc[flower][y])][y]
    out = groups.apply(outliers).dropna()

    if not out.empty:
        outx = []
        outy = []
        for keys in out.index:
            outx.append(keys[0])
            outy.append(out.loc[keys[0]].loc[keys[1]])
    
    qmin = groups.quantile(q=0.00)
    qmax = groups.quantile(q=1.00)
    upper[y] = [min([x1,y1]) for (x1,y1) in zip(list(qmax.loc[:,y]),upper[y])]
    lower[y] = [max([x1,y1]) for (x1,y1) in zip(list(qmin.loc[:,y]),lower[y])]
    
    self.segment(flowerList, upper[y], flowerList, q3[y], line_color="black")
    self.segment(flowerList, lower[y], flowerList, q1[y], line_color="black")

    self.vbar(flowerList, 0.7, q2[y], q3[y], fill_color="#E08E79", line_color="black")
    self.vbar(flowerList, 0.7, q1[y], q2[y], fill_color="#3B8686", line_color="black")
    
    self.rect(flowerList, lower[y], 0.2, 0.01, line_color="black")
    self.rect(flowerList, upper[y], 0.2, 0.01, line_color="black")

    if not out.empty:
        self.circle(outx, outy, size=6, color="#F38630", fill_alpha=0.6)
    
    #label = Label( x=50, y=50, x_units='screen', y_units='screen',
     #              text='ToDo', render_mode='css' )
    #self.add_layout(label)

Figure.vboxplot = vboxplot

In [10]:
def hboxplot( self, source, x, y, **kwargs ):
    if not isinstance(source, pd.DataFrame ):
        raise TypeError("source has to be a pandas DataFrame.")

    flowerList = np.sort(source[y].unique()).tolist()
    df = pd.DataFrame(source)
    
    groups = df.groupby(y)
    q1 = groups.quantile(q=0.25)
    q2 = groups.quantile(q=0.5)
    q3 = groups.quantile(q=0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr
    
    def outliers(group):
        flower = group.name
        return group[(group[x] > upper.loc[flower][x]) | (group[x] < lower.loc[flower][x])][x]
    out = groups.apply(outliers).dropna()

    if not out.empty:
        outx = []
        outy = []
        for keys in out.index:
            outx.append(keys[0])
            outy.append(out.loc[keys[0]].loc[keys[1]])
        
    qmin = groups.quantile(q=0.00)
    qmax = groups.quantile(q=1.00)
    upper[x] = [min([x1,y1]) for (x1,y1) in zip(list(qmax.loc[:,x]),upper[x])]
    lower[x] = [max([x1,y1]) for (x1,y1) in zip(list(qmin.loc[:,x]),lower[x])]
    
    self.segment(upper[x], flowerList, q3[x], flowerList, line_color="black")
    self.segment(lower[x], flowerList, q1[x], flowerList, line_color="black")

    self.hbar(flowerList, 0.7, q2[x], q3[x], fill_color="#E08E79", line_color="black")
    self.hbar(flowerList, 0.7, q1[x], q2[x], fill_color="#3B8686", line_color="black")
    
    self.rect(lower[x], flowerList, 0.01, 0.2, line_color="black")
    self.rect(upper[x], flowerList, 0.01, 0.2, line_color="black")

    if not out.empty:
        self.circle(outy, outx, size=6, color="#F38630", fill_alpha=0.6)
    
    #label = Label( x=50, y=50, x_units='screen', y_units='screen',
     #              text='ToDo', render_mode='css' )
    #self.add_layout(label)

Figure.hboxplot = hboxplot

**<font color="deeppink">Check</font>** your boxplot

In [11]:
p1 = figure( plot_width=400, plot_height=300, y_range=['setosa', 'versicolor', 'virginica'] )
p1.hboxplot( flowers, 'petal_width', 'species' )
p1.xaxis.axis_label = 'petal_width'
p1.yaxis.axis_label = 'species'

p2 = figure( plot_width=400, plot_height=300, x_range=['setosa', 'versicolor', 'virginica'] )
p2.vboxplot( flowers, 'species', 'petal_width' )
p2.yaxis.axis_label = 'petal_width'
p2.xaxis.axis_label = 'species'

show( row(p1,p2))


**<font color="deeppink">Test cases</font>**: Give three scenarios that need testing (bullet points, no implementation required).
    - Test case 1: The groupby clause is applied on the species column. If it contains NAN values, then separate group of NAN   will be created with required a arithematic.
    - Test case 2: Check whether all values are numerical while calculating mean and standard deviation (Ex: values of petal width).
    - Test case 3: Check if the source being passed is null or not.

## Exercise 1 c): Implement a histogram chart

Requirements:
- **Parameters**: The function accepts (at least) the following parameters:
    - **source**: a pandas DataFrameObject that holds the data
    - **x**: variable (name as string) to be represented on the x-axis
    - **nbins**: number of bins (optional argument). If not provided set a meaningful default.
- **Data type**: Provide histograms for categorical and quantitative data.
- **Scaling**: The y-axis shall give probabilities (0,1). Scale the axis to show the full range, e.g., (-0.05,1.05).
- **Calling the histogram**: The function is a class method of Figure and can be called as follows
```python
p = figure()
p.histogram( data, x )
```

Hints:
- Assume that all categorical data has type string. Respective columns in the data can be converted using:
```
df.var = df.var.astype('str')
```

**<font color="deeppink">Implement</font>**

In [12]:
from bokeh.models import Range1d, FactorRange

def histogram( self, source, x, nbins=0, *args, **kwargs ):  
    if not isinstance(source, pd.DataFrame ):
        raise TypeError("source has to be a pandas.DataFrame. Received ", type(df))

    data = source[x]
    dataList = pd.DataFrame(data.tolist(), columns=[x])
    n = len(dataList)
    if (data.values.dtype not in [np.int,np.float]):       
        probDataList = pd.DataFrame({'count': dataList.groupby([x]).size()}).reset_index()      
        nameList = probDataList[x].tolist()
        count = probDataList['count'].tolist()
       
        yaxiscount = np.array(count) / n #bins
        self.vbar( x=nameList, top=yaxiscount, width=0.3, fill_color='#ff69b4', line_color='#e75480')
        self.yaxis.axis_label = 'P('+x+')'
        self.y_range = Range1d(-0.05,1.05);
    else:
        hist, edges = np.histogram(dataList[x], density=True, bins=9)
        self.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="#036564", line_color="#033649")
        self.y_range = Range1d(-0.05,1.05);

   
    #label = Label( x=50, y=50, x_units='screen', y_units='screen',
     #              text='ToDo', render_mode='css' )
    #self.add_layout(label)

Figure.histogram = histogram

**<font color="deeppink">Check</font>** your histogram

In [13]:
var1 = 'sepal_length'
var2 = 'species'
var3 = 'name'

p1 = figure( plot_width=200, plot_height=200 )
p1.histogram( flowers, var1 )
p1.yaxis.axis_label = 'probability'
p1.xaxis.axis_label = var1

labels = np.sort(flowers[var2].unique())
p2 = figure( plot_width=200, plot_height=200, x_range=labels )
p2.histogram( flowers, var2)
p2.xaxis.axis_label = var2

labels = np.sort(autompg[var3].unique())
p3 = figure( plot_width=200, plot_height=200, x_range=labels )
p3.histogram( autompg, var3 )
p3.xaxis.axis_label = var3

show( row( p1, p2, p3 ) )

**<font color="deeppink">Test cases</font>**: Give three scenarios that need testing (bullet points, no implementation required).
1. Ensurity of bin values greater than 0 and natural numbers.
2. How will the program behave if there is a combination of positive and negative values.
3. Check if atleast one of the feature is quantitative or not.

## Exercise 2: Working with SPLOMs

The source code for the generalized scatterplot matrix (SPLOM) is stored in file splom.py. 

Usage:
```
p = splom( df, cols=['var1', 'var2', 'var3'], splom_width=1000 )
show(p)
```

Accepted parameters:
- **source** (req): pandas DataFrame
- **splom_width** (opt): total width/height of the plot.
- **cols** (opt): Array of column names to be used in the plot.
- **x_padding** (opt): additional space for the x-axis labels.
- **y_padding** (opt): additional space for the y-axis labels.

Hint:
- The SPLOM supports some interaction. Select points in the scatterplots and look at the results in the other scatterplots.

In [14]:
%run SPLOM.py

### Exercise 2a): Baseball data

In [15]:
baseball = pd.read_csv( './data/baseball_data.csv')

In [16]:
show( splom( df=baseball, cols=['handedness', 'height', 'weight', 'avg', 'HR'],
             splom_width=1000, x_padding=40, y_padding=80 ) )

categorical attributes ['handedness']


### Analysis of histograms for Baseball data.

Histogram 1: handedness vs probability
- The shape is skewed. It can be seen that most of the players are right handed, followed by left handed and lastly both handed.
- The reason for the shape may be because people are mostly right handed. 
- However, there is no supporting data displayed in other charts to confirm the same.

Histogram 2: distribution of height
- From the distribution it can be observed that, there is a peak but not everything is symmetric.
- This may be due to the fact that people of similar height of a particular range get chosen to play baseball than people shorter or taller compared to the range.
- The distribution of wieght also shows a similiar shape, hence we can conclude that shorter players have lesser weight. 

Histogram 3: distribution of weight
- From the distribution, it can be observed that, there is a peak but more symmetric compared to distribution of height.
- This may be because of players having similar build structure.
- Since the distribution of height also shows a similiar shape, we can say that tall players have more weight and vice versa.

Histogram 4: average distribution
- There are 2 to 4 peaks in the graph.
- This may be because there are too many players with a low average, but on the contrary there are too many players with an average of 0.2 to 0.3.
- The distribution of HR vs Average displays that players with high HR score are most likely to be in 0.2 to 0.3.

Histogram 5: HR distribution
- There is a huge peak on the left side which basically suggest that most of the players score no HR or score little.
- This may be because only good players score decent home runs and also home runs are rarely scored in baseball.
- The average distribution proves the above assumption.


### Exercise 2b): Passengers on the Titanic

Remarks:
- For some passengers age information is missing the `fillna` command replaces those entries with -1. Feal free to make changes to this treating of missing values.
- All data is given as quantitative values. To make distinction of categorical data easier, we turn them into strings.

In [17]:
titanic = pd.read_csv( './data/titanic3.csv')

titanic.pclass = titanic.pclass.astype('str')
titanic.survived = titanic.survived.astype('str')

titanic = titanic.fillna(-1)

p = splom( df=titanic, cols=['pclass', 'survived', 'sex', 'age', 'fare'], splom_width=1000, 
           x_padding=40, y_padding=80 )
show( p )

categorical attributes ['pclass', 'survived', 'sex']


### Analysis of passenger class.

A1. The highest number of people were from the third class, followed by first class. Second class had less number of people.

However, the number of survivors were more in first class (200), followed by surviviors in third class (approximately 190 ) and lastly in second class (approximately 110)

From the data, we find that first class passengers were given more priority while saving lives.

A2. It can be observed that, number of male passengers are more in all passenger classes compared to female passengers. Especially in third class, the male passengers significantly outnumber female passengers.

From the box plots, it is visible that the fare was higher in first class, around median fare of 60 dollars. However, the outliers show that the tickets were purchased at higher prices. In second and third class, the median fare is below 20. 

With respect to age, we find that travellers in first class have a median age of 38, while in second class its about 28. The median age for passengers in third class displayed is about 18.

## Exercise 3: 

### Option 1: Auto MPG

In [18]:
from bokeh.sampledata.autompg import autompg

autompg.cyl = autompg.cyl.astype('str')
autompg.origin = autompg.origin.astype('str')

show( splom( df=autompg, cols=['mpg', 'cyl', 'displ', 'hp', 'weight', 'accel', 'yr', 'origin'],
             splom_width=1000, x_padding=40, y_padding=80 ) )

categorical attributes ['cyl', 'origin']


### Option 2: Iris flowers

In [19]:
from bokeh.sampledata.iris import flowers
p = splom( df=flowers, splom_width=1000, x_padding=40, y_padding=80 )
show( p )

categorical attributes ['species']


### Analyzing data with SPLOMs. Chosen dataset: IRIS 
A1. The data is about 3 different species belonging to a type of Iris plant. We found that, there are about 50 instances of each and contains information about their sepal length, sepal width, petal length and petal width in cms. 

A2. We look at the histogram of sepal_with against sepal_width.
- We found that the data is normally distributed which leads to the implication that 68% of the sepal width is within the range of 2.8 to 3.5cm. 
- Because of this, the unusual thing that we found in this data distribution is when we compared petal width against each other. It produces very skewed distribution with huge amount of petals having width within the range of 0.1 - 0.4 and no petal in the range of 0.6 to 0.9. 
- Furthermore, we found a sharp peak ranging from 1.0 to 1.4 and then there is a gradual decrease. This implies that the petal width of the various species are not having even width. 
- In order to prove this, further investigations of more samples of each species is needed upon which the graph can be cross verified.

 
A3. With the preliminary examination of the dataset, we found that there is a total of 150 samples which are evenly divided among three species namely Setosa, Versicolor and Verginica. Petal length and petal width of setosa is clearly different from that of Versicolor and Verginica, where as it will be difficlut to distinguish the species by looking at the sepal length and sepal width. There also seems to be a bias in the dataset when we look at the histogram of the petal width which leads to a  very skewed distribution. Further investigations with respect to more dataset for each species are needed.